In [ ]:
# clustering 코드1 - perturbation(word embedding x) 적용
# 130 서버 gpu 1 사용중

%load_ext autoreload
%autoreload 2
CUDA_VISIBLE_DEVICES=1

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import argparse
import torch


#from tqdm.notebook import tqdm, trange
from tqdm import tqdm, trange

from diffmask.models.sentiment_classification_sst_hhs import (
    BertSentimentClassificationSST,
    BertSentimentClassificationSST_hhs, # 미분(CLS 벡터)
    BertSentimentClassificationSST_hhs2, # 드랍아웃 레이어
    BertSentimentClassificationSST_hhs3, # 미분(word embedding)
    BertSentimentClassificationSST_hjy, # cls embedding
    MyDataset,
    my_collate_fn,
    my_collate_fn_rationale,
    load_sst
)

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--gpu", type=str, default="1")
    parser.add_argument("--seed", type=int, default=0)
    parser.add_argument(
        "--val_filename",
        type=str,
        # default="datasets/eSNLI/esnli_test.csv"
        # default="datasets/coco_test/train_api.txt"
        # default="datasets/CoT/snli/clustering/split_1.txt"
        default="datasets/CoT/snli/snli_label_changed.txt"
        # default="datasets/sci_chatgpt/test_data/500_mnli_mis.txt"
        
    )
    parser.add_argument(
        "--val_rationale",
        type=str,
        default="datasets/eSNLI/esnli_test.rationale_idx"
    )
    
    parser.add_argument(
        "--token_cls", 
        action='store_true'
    )
    
    parser.add_argument(
        "--model_path",
        type=str,
        default="outputs/coco-bert-hjy_snli_15label_new_split/epoch=11-val_acc=0.9656-val_f1=0.9414.ckpt"
    )
    
    parser.add_argument(
        "--dataset",
        type=str,
        default="coco", # coco, esnli
    )

    hparams, _ = parser.parse_known_args()
    
    # 환경 변수 설정
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = hparams.gpu

    # GPU 할당
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)

model = BertSentimentClassificationSST_hjy.load_from_checkpoint(hparams.model_path).to(device) # 내가 원하는 형태로 output만 뽑기
model.hparams.model_path = hparams.model_path

model.freeze()
model.prepare_data()
print("Current CUDA device index:", torch.cuda.current_device())

In [ ]:
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
import torch.nn.functional as F

# softmax with temperature scaling 적용
def softmax_with_temperature(logits, T):
    logits = logits / T  # Temperature scaling
    max_logits = torch.max(logits, dim=1, keepdim=True)[0]
    exp_logits = torch.exp(logits - max_logits)
    sum_exp_logits = torch.sum(exp_logits, dim=1, keepdim=True)
    y = exp_logits / sum_exp_logits
    return y

T = 700 
epsilon = 0.033
probability_threshold = 0.071486

# val_dataset 및 val_dataloader 초기화
val_dataset, _ = load_sst(
                hparams.val_filename, None, hparams.dataset, model.hparams.num_labels, hparams.val_rationale, model.hparams.token_cls
            )
val_dataloader = torch.utils.data.DataLoader(
            val_dataset, batch_size=1, collate_fn=my_collate_fn_rationale, num_workers=8
        )

selected_clustering_values = []
selected_clustering_indices = []
selected_premises = []
selected_hypotheses = []
selected_cls_embeddings = []  # cls embedding을 저장할 리스트 추가

# 입력데이터를 모델에 전달
for i, batch in tqdm(enumerate(model.val_dataloader()), total=len(model.val_dataset) // model.hparams.batch_size): # Coco
    model.eval()
    inputs = model.tokenizer.batch_encode_plus(batch[0], pad_to_max_length=True, return_tensors='pt').to(device)
    input_ids = inputs['input_ids']
    mask = inputs['attention_mask']
    token_type_ids = inputs['token_type_ids']
    labels = batch[1].to(device)
    
    logits, new_logits, cls_embedding = model.forward(input_ids, mask, token_type_ids, temperature=T, epsilon=epsilon)  # cls_embedding 추가

    model.zero_grad()
    
    # new_logits에 대해 소프트맥스 함수를 적용
    softmaxed_clustering_output = F.softmax(new_logits, dim=1)  # 이 코드로 진행
    
    max_values_clustering, max_indices_clustering = torch.max(softmaxed_clustering_output, dim=1)
    
    for j in range(len(max_values_clustering)):
        if max_values_clustering[j] <= probability_threshold:
            selected_clustering_indices.append(max_indices_clustering[j].detach().cpu().numpy())
            selected_clustering_values.append(max_values_clustering[j].detach().cpu().numpy())
            
            # 원래의 문장으로 복원
            premise, hypothesis = batch[0][j]
            
            selected_premises.append(premise.strip())
            selected_hypotheses.append(hypothesis.strip())
            
            # cls embedding 추가
            selected_cls_embeddings.append(cls_embedding[j].detach().cpu().numpy())  # CLS 토큰 임베딩 추가

# 데이터 프레임 생성 및 엑셀로 저장
results_df = pd.DataFrame({
    'Indices': selected_clustering_indices,
    'Probabilities': selected_clustering_values,
    'Premise': selected_premises,
    'Hypothesis': selected_hypotheses,
    'CLS_Embedding': selected_cls_embeddings  # cls embedding 추가
})

# Excel 파일로 저장 (파일 경로에 따라 수정 필요)
results_df.to_excel('/home/hjy/Rationale_Extraction_using_Diffmask-main/Rationale_Extraction_using_Diffmask-main/clustering/output_sort_51914_cls.xlsx', index=False)

# 저장된 행의 개수 출력
num_selected_rows = len(selected_clustering_values)
print(f"Number of rows saved: {num_selected_rows}")


In [ ]:
## 여기부터 돌려보기 걸리는 시간 5000개 기준 36s

import pandas as pd
import os
import glob

# 엑셀파일 경로
cluster_output_dir = '/home/hjy/Rationale_Extraction_using_Diffmask-main/Rationale_Extraction_using_Diffmask-main/clustering/cluster_output_5000/'

# 모든 클러스터링 된 엑셀 파일 읽기
cluster_files = glob.glob(os.path.join(cluster_output_dir, 'output_cluster_*.xlsx'))

# 각 클러스터 파일에 대해 'Indices'의 빈도수 계산 및 결과 저장
results = []
for file in cluster_files:
    df = pd.read_excel(file)
    index_counts = df['Indices'].value_counts()
    most_common_index = index_counts.idxmax()
    most_common_index_count = index_counts.max()
    total_indices = len(df)
    most_common_index_ratio = most_common_index_count / total_indices * 100
    
    results.append({
        'file' : os.path.basename(file),
        'most_common_index' : most_common_index,
        'most_common_index_count' : most_common_index_count,
        'total_indices' : total_indices,
        'most_common_index_ratio' : most_common_index_ratio
    })
    
# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(results)

# 결과 출력
print(results_df)

# 빈도수가 높은 순으로 정렬
results_df = results_df.sort_values(by='most_common_index_ratio', ascending=False)

# 결과를 엑셀 파일로 저장
results_df.to_excel('/home/hjy/Rationale_Extraction_using_Diffmask-main/Rationale_Extraction_using_Diffmask-main/clustering/cluster_analysis_5000_results.xlsx', index=False)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import re

# 엑셀 파일 불러오기
input_file = '/home/hjy/Rationale_Extraction_using_Diffmask-main/Rationale_Extraction_using_Diffmask-main/clustering/output_sort_51914_cls.xlsx'
results_df = pd.read_excel(input_file)

# CLS 임베딩 데이터 전처리 함수
def preprocess_embedding(embedding_str):
    # 쉼표를 추가하여 올바른 리스트 형식으로 변환
    embedding_str = re.sub(r'(?<=\d)\s+(?=-?\d)', ', ', embedding_str.strip())
    return np.array(eval(embedding_str))

# CLS 임베딩 데이터 추출 및 전처리
cls_embeddings = results_df['CLS_Embedding'].apply(preprocess_embedding).tolist()
cls_embeddings = np.stack(cls_embeddings)

# K-means 클러스터링 적용
num_clusters = 1000  # 원하는 클러스터 개수 설정
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
clusters = kmeans.fit_predict(cls_embeddings)

# 클러스터 정보 추가
results_df['Cluster'] = clusters

# 클러스터별 문장쌍 추출 및 엑셀 파일로 저장
for cluster_num in range(num_clusters):
    cluster_df = results_df[results_df['Cluster'] == cluster_num]
    cluster_df.to_excel(f'/home/hjy/Rationale_Extraction_using_Diffmask-main/Rationale_Extraction_using_Diffmask-main/clustering/cluster_output_1000/output_cluster_{cluster_num}.xlsx', index=False)

# 각 클러스터에 속하는 문장 쌍의 개수를 계산
cluster_sizes = results_df['Cluster'].value_counts().sort_values(ascending=False)

# 각 클러스터에 속하는 문장 쌍의 개수를 출력 (문장 수 기준으로 정렬)
print(f"Number of clusters: {num_clusters}")
for cluster_num, size in cluster_sizes.iteritems():
    print(f"Cluster {cluster_num}: {size} rows")

# t-SNE 결과를 사용하여 2차원으로 축소
perplexity = min(30, len(cls_embeddings) // 2)
tsne = TSNE(n_components=2, random_state=0, perplexity=perplexity)
tsne_result = tsne.fit_transform(cls_embeddings)

# t-SNE 결과를 데이터프레임에 추가
results_df['tsne-2d-one'] = tsne_result[:, 0]
results_df['tsne-2d-two'] = tsne_result[:, 1]

# 시각화
plt.figure(figsize=(16,10))
scatter = plt.scatter(results_df['tsne-2d-one'], results_df['tsne-2d-two'], c=results_df['Cluster'], cmap='viridis', alpha=0.6)
plt.colorbar(scatter, label='Cluster')

# # 클러스터 중심에 클러스터 번호 추가 (옵션)
# cluster_centers = kmeans.cluster_centers_
# tsne_cluster_centers = tsne.fit_transform(cluster_centers)
# for i, txt in enumerate(range(num_clusters)):
#     plt.annotate(txt, (tsne_cluster_centers[i, 0], tsne_cluster_centers[i, 1]), fontsize=9, fontweight='bold')

plt.title('t-SNE visualization of CLS Embeddings with KMeans Clustering')
plt.xlabel('t-SNE Dimension 1')
plt.ylabel('t-SNE Dimension 2')
plt.show()


In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import re

# 엑셀 파일 불러오기
input_file = '/home/hjy/Rationale_Extraction_using_Diffmask-main/Rationale_Extraction_using_Diffmask-main/clustering/output_sort_51914_cls.xlsx'
results_df = pd.read_excel(input_file)

# CLS 임베딩 데이터 전처리 함수
def preprocess_embedding(embedding_str):
    # 쉼표를 추가하여 올바른 리스트 형식으로 변환
    embedding_str = re.sub(r'(?<=\d)\s+(?=-?\d)', ', ', embedding_str.strip())
    return np.array(eval(embedding_str))

# CLS 임베딩 데이터 추출 및 전처리
cls_embeddings = results_df['CLS_Embedding'].apply(preprocess_embedding).tolist()
cls_embeddings = np.stack(cls_embeddings)

# K-means 클러스터링 적용
num_clusters = 5000  # 원하는 클러스터 개수 설정
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
clusters = kmeans.fit_predict(cls_embeddings)

# 클러스터 정보 추가
results_df['Cluster'] = clusters

# 클러스터별 문장쌍 추출 및 엑셀 파일로 저장
for cluster_num in range(num_clusters):
    cluster_df = results_df[results_df['Cluster'] == cluster_num]
    cluster_df.to_excel(f'/home/hjy/Rationale_Extraction_using_Diffmask-main/Rationale_Extraction_using_Diffmask-main/clustering/cluster_output_5000/output_cluster_{cluster_num}.xlsx', index=False)

# 각 클러스터에 속하는 문장 쌍의 개수를 계산
cluster_sizes = results_df['Cluster'].value_counts().sort_values(ascending=False)

# 각 클러스터에 속하는 문장 쌍의 개수를 출력 (문장 수 기준으로 정렬)
print(f"Number of clusters: {num_clusters}")
for cluster_num, size in cluster_sizes.iteritems():
    print(f"Cluster {cluster_num}: {size} rows")

# t-SNE 결과를 사용하여 2차원으로 축소
perplexity = min(30, len(cls_embeddings) // 2)
tsne = TSNE(n_components=2, random_state=0, perplexity=perplexity)
tsne_result = tsne.fit_transform(cls_embeddings)

# t-SNE 결과를 데이터프레임에 추가
results_df['tsne-2d-one'] = tsne_result[:, 0]
results_df['tsne-2d-two'] = tsne_result[:, 1]

# 시각화
plt.figure(figsize=(16,10))
scatter = plt.scatter(results_df['tsne-2d-one'], results_df['tsne-2d-two'], c=results_df['Cluster'], cmap='viridis', alpha=0.6)
plt.colorbar(scatter, label='Cluster')

# # 클러스터 중심에 클러스터 번호 추가 (옵션)
# cluster_centers = kmeans.cluster_centers_
# tsne_cluster_centers = tsne.fit_transform(cluster_centers)
# for i, txt in enumerate(range(num_clusters)):
#     plt.annotate(txt, (tsne_cluster_centers[i, 0], tsne_cluster_centers[i, 1]), fontsize=9, fontweight='bold')

plt.title('t-SNE visualization of CLS Embeddings with KMeans Clustering')
plt.xlabel('t-SNE Dimension 1')
plt.ylabel('t-SNE Dimension 2')
plt.show()


In [ ]:
## 여기부터 돌려보기 걸리는 시간 5000개 기준 36s

import pandas as pd
import os
import glob

# 엑셀파일 경로
cluster_output_dir = '/home/hjy/Rationale_Extraction_using_Diffmask-main/Rationale_Extraction_using_Diffmask-main/clustering/cluster_output_10000/'

# 모든 클러스터링 된 엑셀 파일 읽기
cluster_files = glob.glob(os.path.join(cluster_output_dir, 'output_cluster_*.xlsx'))

# 각 클러스터 파일에 대해 'Indices'의 빈도수 계산 및 결과 저장
results = []
for file in cluster_files:
    df = pd.read_excel(file)
    index_counts = df['Indices'].value_counts()
    most_common_index = index_counts.idxmax()
    most_common_index_count = index_counts.max()
    total_indices = len(df)
    most_common_index_ratio = most_common_index_count / total_indices * 100
    
    results.append({
        'file' : os.path.basename(file),
        'most_common_index' : most_common_index,
        'most_common_index_count' : most_common_index_count,
        'total_indices' : total_indices,
        'most_common_index_ratio' : most_common_index_ratio
    })
    
# 결과를 데이터프레임으로 변환
results_df = pd.DataFrame(results)

# 결과 출력
print(results_df)

# 빈도수가 높은 순으로 정렬
results_df = results_df.sort_values(by='most_common_index_ratio', ascending=False)

# 결과를 엑셀 파일로 저장
results_df.to_excel('/home/hjy/Rationale_Extraction_using_Diffmask-main/Rationale_Extraction_using_Diffmask-main/clustering/cluster_analysis_10000_results.xlsx', index=False)